# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../WeatherPY/output_data/clean_cities_data_2.csv"
cities = pd.read_csv(file_path)
cities_df = pd.DataFrame(cities)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,adrar,20.5022,-10.0711,80.08,7.0,0.0,10.74,MR,1.616788e+09
1,chokurdakh,70.6333,147.9167,-31.85,100.0,76.0,4.47,RU,1.616788e+09
2,laguna,38.4210,-121.4238,69.01,24.0,1.0,16.11,US,1.616788e+09
3,mehamn,71.0357,27.8492,37.40,64.0,0.0,35.68,NO,1.616788e+09
4,lleida,41.6167,0.6167,62.01,67.0,0.0,3.44,ES,1.616788e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

#store humidity
humidity = cities_df["Humidity"].astype(float)

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [31]:
# Customize the size of the figure

fig = gmaps.figure(layout=figure_layout)

#add heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#weather criteria: temperature between 60-75; wind speed less than 5mph; 10% cloudiness or less 
ideal_df = cities_df.loc[(cities_df["Max Temp"] >= 60) & (cities_df["Max Temp"] < 75) & (cities_df["Wind Speed"] < 5) & (cities_df["Cloudiness"] < 10)]
ideal_df = ideal_df.dropna()
ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,lleida,41.6167,0.6167,62.01,67.0,0.0,3.44,ES,1.616788e+09
47,mackay,-21.1500,149.2000,71.60,88.0,1.0,4.61,AU,1.616788e+09
99,beloha,-25.1667,45.0500,73.56,83.0,1.0,3.36,MG,1.616788e+09
122,saldanha,-33.0117,17.9442,66.20,72.0,0.0,3.44,ZA,1.616788e+09
182,thakurgaon,26.0333,88.4667,71.91,13.0,0.0,3.76,BD,1.616788e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
ideal_df["Hotel Name"] = ""
hotel_df = ideal_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,lleida,41.6167,0.6167,62.01,67.0,0.0,3.44,ES,1.616788e+09,
47,mackay,-21.1500,149.2000,71.60,88.0,1.0,4.61,AU,1.616788e+09,
99,beloha,-25.1667,45.0500,73.56,83.0,1.0,3.36,MG,1.616788e+09,
122,saldanha,-33.0117,17.9442,66.20,72.0,0.0,3.44,ZA,1.616788e+09,
182,thakurgaon,26.0333,88.4667,71.91,13.0,0.0,3.76,BD,1.616788e+09,


In [34]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    data = requests.get(base_url, params=params).json()

    # try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df = hotel_df.dropna()
hotel_df

Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,lleida,41.6167,0.6167,62.01,67.0,0.0,3.44,ES,1.616788e+09,Hotel Avenida El Morell
47,mackay,-21.1500,149.2000,71.60,88.0,1.0,4.61,AU,1.616788e+09,Ocean International Hotel
99,beloha,-25.1667,45.0500,73.56,83.0,1.0,3.36,MG,1.616788e+09,
122,saldanha,-33.0117,17.9442,66.20,72.0,0.0,3.44,ZA,1.616788e+09,Windtown Lagoon Hotel
182,thakurgaon,26.0333,88.4667,71.91,13.0,0.0,3.76,BD,1.616788e+09,Hotel Address Inn
245,tura,25.5198,90.2201,71.83,13.0,0.0,3.00,IN,1.616789e+09,"Dongrama Home Stay, Tura Meghalaya"
253,faya,18.3851,42.4509,66.20,63.0,0.0,1.39,SA,1.616789e+09,Sarawat Park Hotel & Resort
313,zafra,38.4167,-6.4167,60.80,55.0,0.0,4.61,ES,1.616789e+09,Hotel La Muralla
375,salinas,36.6777,-121.6555,63.00,45.0,1.0,3.44,US,1.616789e+09,Post Ranch Inn
391,sakaraha,-22.9000,44.5333,68.68,91.0,6.0,2.55,MG,1.616789e+09,Hôtel 2000


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)

# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))